In [127]:
%matplotlib inline

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import random
import cv2
from sklearn import metrics
from tqdm.notebook import trange, tqdm
from sklearn import preprocessing
import sys
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import glob

import warnings
warnings.filterwarnings('ignore')

<b>Useful source:</b> https://towardsdatascience.com/wtf-is-image-classification-8e78a8235acb (general basic knowledge how to start dealing with computer vision), https://towardsdatascience.com/image-pre-processing-c1aec0be3edf (how to pre-process data), https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/ (how cnn works), https://ujjwalkarn.me/2016/08/09/quick-intro-neural-networks/ (quick introduction to nn in general), https://towardsdatascience.com/image-processing-techniques-for-computer-vision-11f92f511e21 (images preprocessing and data augmentation)

### Operations used:

#### 1) Standarization

Resizing every image to exactly same shape

#### 2) Denoise 
Smoothing images to remove noise we do not want them to appear on our photos. Noise is somrthing unwanted wich do not bring any information, moreover it could disrupt our model in recognizing our target. Our action helps enhance image structures.

#### 3) Grayscale

Due to only one color channel in grayscale photos processing is at least 3 times faster. To gender recognition photos color in my opinion is not useful. When we will be predicting ethnicity in the next project's stage, color of photos could be useful. I will find out it during work on the second stage of project, because it's my first step to understand computer vision, so my knowledge about it is a bit basic now.

In computer vision is something called thumb rule to recognize color importance in a project. It refers to look at the picture with human eye and try to recognize object we are looking for in a grayscaled photo. If we can, color it's not important. I think that we are able to approximate human age without colors attendance at the picture, so in my opinion this rule is fullfiled in this case.

<b>Useful source:</b> https://manningbooks.medium.com/the-computer-vision-pipeline-part-3-image-preprocessing-c662094835ec

#### 4) Normalization
Firstly normalization of photos' pixels helps us avoid situation where big pixels dominate smaller ones because of huge range (here 0 to 255). Secondly many algorithms behave well on normalized data rather than on not normalized.

<b>Useful source:</b> https://medium.com/@kharikri/image-processing-tips-for-computer-vision-and-deep-learning-tasks-e5247ec94f3

### Loading images and data manipulations

In [128]:
# resizing variables
height, width = 100, 100

In [129]:
def load_photo_and_preprocess(
    path_to_images, 
    class_1, 
    class_2,
    height,
    width,
):
    X = []
    y = []
    os.chdir(path_to_images)
    for img_name in tqdm(os.listdir(path_to_images)):
        img = cv2.imread(img_name)   
        # Standarization
        img = cv2.resize(img, (height, width), interpolation = cv2.INTER_AREA)
        # Denoise
        img = cv2.GaussianBlur(img, (5, 5), 0)
        # Grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Normalization
        img = img/255
        X.append(img)
        # Getting labels
        if img_name.split('.')[0] == class_1:
            y.append(0) 
        elif img_name.split('.')[0] == class_2:
            y.append(1) 

    # transforming label to appropriate format
    y = k.utils.to_categorical(y, num_classes=2)
    
    # list to array transform
    X = np.array(X)
    
    return X, y

In [130]:
X, y = load_photo_and_preprocess(
    '/Users/alexdrozdz/JupyterNotebook/Cats_dogs/Data',
    'cat',
    'dog',
    height,
    width,
)

In [131]:
print("Image shape: ", X[0].shape)

Image shape:  (100, 100)


In [132]:
def plot_image(index, X, y, class_1, class_2):
    # plot one random image
    plt.imshow(X[index], cmap='gray')
    plt.show()
    if y[index][0] == 1:
        label = class_1
    elif y[index][0] == 0:
        label = class_2
    print(f"Label: {label}")

Label: dog


In [133]:
plot_image(np.random.randint(len(X)), X, y, 'cat', 'dog')

#### Train test split

In [134]:
print(f"Dataset length: {len(X)}")

Dataset length: 25000


In [135]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state = 1234
)

X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, shuffle=True, random_state = 1234
)

In [136]:
data_inf_dict = {
    'X_train size': len(X_train),
    'X_test size': len(X_test),
    'X_val size': len(X_val),
    'y_train size': len(y_train),
    'y_test size': len(y_test),
    'y_val size': len(y_val),
}

data_inf_dict

{'X_train size': 20000,
 'X_test size': 2500,
 'X_val size': 2500,
 'y_train size': 20000,
 'y_test size': 2500,
 'y_val size': 2500}

In [137]:
def count_ratio(y, name):
    class_0 = []
    class_1 = []
    for element in y:
        if element[0] == 0:
            class_0.append(1)
        if element[0] == 1:
            class_1.append(1)        
            
    ratio = str(
        round(len(class_0)/(len(class_1)+len(class_0))*100,2)
    ) + '/' + str(
        100 - round(len(class_0)/(len(class_1)+len(class_0))*100,2)
    )
    
    print(f"{name} balance ratio: {ratio}")

In [138]:
count_ratio(y_train, 'y_train')
count_ratio(y_test, 'y_test')
count_ratio(y_val, 'y_val')

y_train balance ratio: 49.9/50.1
y_test balance ratio: 49.88/50.12
y_val balance ratio: 50.96/49.04


Classes in datasets are almost perfectly balanced after shuffle

### Convolutional Neural Network

#### Convolution layer
Used to extract (filter) certain features from group of pixels

#### Filters
A set of weights - filter - to extract local features in convolutional layer

The goal of using filters is to modify or enhance image properties and/or to extract valuable information from the pictures such as edges, corners, and blobs.

Goals:
- Extract out important information.
- Remove noise or unwanted elements.
- Enhance the image.
- Give a certain look

#### Non-linearity layer
Often ReLU. Allows to deal with non-linearity data. ReLU change every negative value to 0.

#### Pooling layer
Using to reduce dimensionality of input layer. 

#### 3 steps of feature learning:

1. Learn features in input image through convolution
2. Introduce non-linearity through activation function (real-world data is non-linear!)
3. Reduce dimensionality and preserve spatial invariance with pooling

#### Classification:
After these 3 steps the next part of CNN is classification: 
- CONV and POOL layers output high-level features of input
- Fully connected layer uses these features for classyfing input image
- Express output as probability of image belonging to a particular class

output features -> Fully Conected Layer (each neuron linked with each neuron, another name: Dense Layer) which output probability of image belongs to particulr class (softmax/sigmoid function)

<b>Useful source:</b> https://ai.stanford.edu/~syyeung/cvweb/tutorial1.html, https://medium.com/towards-artificial-intelligence/image-filtering-dfe29581f859, https://www.youtube.com/watch?v=iaSUYvmCekI

In [139]:
X_train = np.expand_dims(X_train, -1)
X_val = np.expand_dims(X_val, -1)
X_test = np.expand_dims(X_test, -1)

##### Batch:
We can’t pass the entire dataset into the neural net at once. So, it's a need to divide dataset into parts, and batch size is a size of these parts.

##### Epochs:
One epoch means one pass of dataset forward and backward (computing and then weights adjusting).

##### Iterations:
Iterations is the number of batches needed to complete one epoch. 1000 training examples could be divided into baches of 100, then we need 10 iterations to end one epoch.

<b>Useful source:</b> https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

In [140]:
input_shape = (height, width, 1)
batch_size = 32
epochs = 10
classes = 2

In [141]:
# cnn model initialization
model = Sequential([
    k.layers.Conv2D(
        filters=32, 
        kernel_size=(3,3), 
        padding='same', 
        activation='relu', 
        input_shape=input_shape,
    ),
    k.layers.MaxPooling2D(
        pool_size=(2,2)
    ),
    k.layers.Dropout(0.5),
    k.layers.BatchNormalization(),
    k.layers.Conv2D(
        filters=64, 
        kernel_size=(3,3), 
        padding='same', 
        activation='relu'
    ),
    k.layers.MaxPooling2D(
        pool_size=(2,2)
    ), 
    k.layers.Dropout(0.5),
    k.layers.BatchNormalization(),
    k.layers.Conv2D(
        filters=64, 
        kernel_size=(3,3), 
        padding='same', 
        activation='relu'
    ),
    k.layers.MaxPooling2D(
        pool_size=(2,2)
    ), 
    k.layers.Dropout(0.5),
    k.layers.BatchNormalization(),
    # flatten converts our 3D feature maps to 1D feature vectors
    k.layers.Flatten(),
    k.layers.Dense(
        128, 
        activation='relu'
    ),
    k.layers.Dense(
        classes,
        activation='sigmoid'
    )
])

In [142]:
# build model
opt = k.optimizers.Adam(lr = 5e-3)
model.compile( 
    loss = "binary_crossentropy", 
    optimizer = opt, 
    metrics=['accuracy']
)

In [143]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 100, 100, 32)      320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 50, 50, 32)        128       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 25, 25, 64)       

In [144]:
# early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

In [145]:
# train the model
model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[callback]
)

Epoch 1/10
625/625 [==============================] - 154s 247ms/step - loss: 0.6933 - accuracy: 0.5917 - val_loss: 0.9819 - val_accuracy: 0.4968
Epoch 2/10
625/625 [==============================] - 148s 237ms/step - loss: 0.6269 - accuracy: 0.6471 - val_loss: 0.6599 - val_accuracy: 0.5908
Epoch 3/10
625/625 [==============================] - 152s 244ms/step - loss: 0.6030 - accuracy: 0.6749 - val_loss: 0.6334 - val_accuracy: 0.6340
Epoch 4/10
625/625 [==============================] - 153s 244ms/step - loss: 0.5874 - accuracy: 0.6849 - val_loss: 0.7811 - val_accuracy: 0.5108
Epoch 5/10
625/625 [==============================] - 151s 241ms/step - loss: 0.5649 - accuracy: 0.7038 - val_loss: 0.7596 - val_accuracy: 0.6100
Epoch 6/10
625/625 [==============================] - 149s 239ms/step - loss: 0.5416 - accuracy: 0.7232 - val_loss: 0.5819 - val_accuracy: 0.6988
Epoch 7/10
625/625 [==============================] - 160s 256ms/step - loss: 0.5327 - accuracy: 0.7308 - val_loss: 0.6887 -

In [146]:
score = model.evaluate(X_test, y_test, verbose=0)

In [147]:
print(f"Model accuracy: {round(score[1]*100,2)}%")

Model accuracy: 70.4%


#### Predicition on my private dog photo

In [148]:
X_predict, y_predict = load_photo_and_preprocess(
    '/Users/alexdrozdz/JupyterNotebook/Cats_dogs/my_dog',
    'cat',
    'dog',
    height,
    width,
)

Label: dog


In [149]:
plot_image(0, X_predict, y_predict, 'cat', 'dog')

In [150]:
X_predict = np.expand_dims(X_predict, -1)

In [153]:
prediction = model.predict(X_predict)

In [154]:
if prediction[0][1] > 0.5:
    print(
        f"Hey, that's right! Of course it's a dog! \nDog probability: {round(prediction[0][1]*100,2)}%, Cat probability: {round(prediction[0][0]*100,2)}%. \nBtw his name is Billy :)"
    )
elif prediction[0][1] < 0.5:
    print(
        f"Egh prediction is bad! For a model it's a cat \nDog probability: {round(prediction[0][1]*100,2)}%, Cat probability: {round(prediction[0][0]*100,2)}%"
    )

Hey, that's right! Of course it's a dog! 
Dog probability: 83.63%, Cat probability: 16.36%. 
Btw his name is Billy :)
